# SQL Insight

In [1]:
import pandas as pd
import sqlite3

# Load the cleaned dataset
df = pd.read_csv("youtube_with_sentiment.csv")

# Helper function to find column name by keyword
def find_col(keyword):
    for col in df.columns:
        if keyword.lower() in col.lower():
            return col
    return None

# Detect relevant column names
views_col = find_col("view")
likes_col = find_col("like")
comments_col = find_col("comment")
title_col = find_col("title")
category_col = find_col("category")
tags_col = find_col("tag")
trending_col = find_col("trending")
sentiment_col = find_col("title_sentiment_label")

# Create SQLite in-memory database
conn = sqlite3.connect(":memory:")
df.to_sql("youtube", conn, index=False, if_exists='replace')
print("Dataset loaded into SQLite memory database as 'youtube' table.\n")

# Query 1: Top 10 Categories by Average Views
if category_col and views_col:
    query1 = f"""
    SELECT "{category_col}" AS category, AVG("{views_col}") AS avg_views
    FROM youtube
    GROUP BY "{category_col}"
    ORDER BY avg_views DESC
    LIMIT 10;
    """
    print("Top 10 Categories by Average Views:\n")
    print(pd.read_sql(query1, conn), "\n")
else:
    print("Category or views column not found. Skipping Query 1.\n")

# Query 2: Top 10 Most Viewed Videos
if title_col and views_col and likes_col and comments_col:
    query2 = f"""
    SELECT "{title_col}" AS title, "{views_col}" AS views, "{likes_col}" AS likes, "{comments_col}" AS comment_count
    FROM youtube
    ORDER BY "{views_col}" DESC
    LIMIT 10;
    """
    print("Top 10 Most Viewed Videos:\n")
    print(pd.read_sql(query2, conn), "\n")
else:
    print("Necessary columns not found for Query 2.\n")

# Query 3: Daily Count of Trending Videos
if trending_col:
    query3 = f"""
    SELECT "{trending_col}" AS trending_date, COUNT(*) AS videos_trending
    FROM youtube
    GROUP BY "{trending_col}"
    ORDER BY "{trending_col}";
    """
    print("Daily Trending Video Count:\n")
    print(pd.read_sql(query3, conn), "\n")
else:
    print("Trending date column not found. Skipping Query 3.\n")

# Query 4: Average Like Ratio by Sentiment
if sentiment_col and views_col and likes_col:
    query4 = f"""
    SELECT "{sentiment_col}" AS sentiment, AVG("{likes_col}" * 1.0 / NULLIF("{views_col}", 0)) AS avg_like_ratio
    FROM youtube
    WHERE "{sentiment_col}" IS NOT NULL
    GROUP BY "{sentiment_col}";
    """
    print("Average Like Ratio by Title Sentiment:\n")
    print(pd.read_sql(query4, conn), "\n")
else:
    print("Necessary columns not found for Query 4.\n")

# Query 5: Most Frequently Used Tags (Optional)
if tags_col:
    query5 = f"""
    SELECT "{tags_col}" AS tags, COUNT(*) as count
    FROM youtube
    GROUP BY "{tags_col}"
    ORDER BY count DESC
    LIMIT 10;
    """
    print("Most Frequently Used Tags:\n")
    print(pd.read_sql(query5, conn), "\n")
else:
    print("Tags column not found. Skipping Query 5.\n")

# Close connection
conn.close()
print("SQL connection closed.")

Dataset loaded into SQLite memory database as 'youtube' table.

Top 10 Categories by Average Views:

   category     avg_views
0        25  27799.833333
1        24  23247.200000
2        28  18291.466667
3        27   6513.750000
4         1   4040.000000
5        20   1851.000000
6        17   1805.000000
7        22    962.720930
8        19    843.000000
9        23    515.000000 

Top 10 Most Viewed Videos:

                          title   views   likes  comment_count
0              PsychedSubstance  526243  7046.0         1706.0
1              PsychedSubstance  148681  3698.0         1212.0
2           Black Pigeon Speaks  108974  4329.0         3672.0
3    The Natural Way of Healing   59255   870.0          523.0
4                 Your Mate Tom   35928   323.0          178.0
5                  Secular Talk   28167   868.0          338.0
6  The Point with Ana Kasparian   27571   358.0          349.0
7                Young Morpheus   18908   200.0           80.0
8      Oracle Fr